# Emissions to forcing for ozone

Keep the AR6 treatment in FaIR v1.6.2+ and remove all previous incarnations.

We'll calculate the GHG concentrations from emissions first, and then use both to calculate ozone forcing along with the temperature feedback.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as pl

from fair21.forcing.ozone import thornhill_skeie
from fair21.constants.gases import molwt, burden_per_emission, lifetime
from fair21.defaults import slcf_list, gas_list
from fair21.defaults.gases import (
    partition_fraction,
    pre_industrial_concentration,
    natural_emissions_adjustment,
    iirf_0,
    iirf_cumulative,
    iirf_temperature,
    iirf_airborne,
    iirf_horizon
)
from fair21.gas_cycle import (
    calculate_g,
    calculate_alpha
)
from fair21.gas_cycle.forward import step_concentration

In [ ]:
# grab some emissions
emissions = {}
df = pd.read_csv('../data/rcmip/rcmip-emissions-annual-means-v5-1-0.csv')
for specie in gas_list + slcf_list:
    specie_rcmip_name = specie.replace("-", "")
    emissions[specie] = df.loc[
        (df['Scenario']=='ssp585') & (df['Variable'].str.endswith("|"+specie_rcmip_name)) & (df['Region']=='World'), '1750':
    ].interpolate(axis=1).values.squeeze()

# CO2 and N2O units need to behave
emissions["CO2"] = emissions["CO2"] / 1000
emissions["N2O"] = emissions["N2O"] / 1000

In [ ]:
# initialise dicts for outputs. These will eventually be moved to the forward model
concentration = {}
airborne_emissions = {}
cumulative_emissions = {}
effective_radiative_forcing = {}

n_timesteps = len(emissions[specie])

In [ ]:
# grab indicative temperature projections
df = pd.read_csv('../data/rcmip-phase2/rcmip-phase2-fair162-ssp585-mean-temperature.csv')
ssp585_temperature_rfmip = df['temperature'].values

In [ ]:
concentration['CO2'] = np.ones(n_timesteps) * np.nan

co2_boxes = 0
airborne_emissions = 0
cumulative_emissions['CO2'] = np.cumsum(emissions['CO2'])

g0, g1 = calculate_g(lifetime['CO2'], partition_fraction=partition_fraction['CO2'])

for i in range(n_timesteps):
    alpha_lifetime = calculate_alpha(
        cumulative_emissions['CO2'][i],
        airborne_emissions,
        ssp585_temperature_rfmip[i],
        iirf_0["CO2"],
        iirf_cumulative["CO2"],
        iirf_temperature["CO2"],
        iirf_airborne["CO2"],
        g0,
        g1,
    )
    concentration['CO2'][i], co2_boxes, airborne_emissions = step_concentration(
        emissions['CO2'][i], 
        co2_boxes,
        airborne_emissions, 
        burden_per_emission['CO2'],
        lifetime['CO2'],
        alpha_lifetime=alpha_lifetime,
        pre_industrial_concentration=pre_industrial_concentration['CO2'],
        timestep=1,
        partition_fraction=partition_fraction['CO2']
    )

In [ ]:
concentration['CH4'] = np.ones(n_timesteps) * np.nan

gas_boxes = 0
airborne_emissions = 0
concentration['CH4'][0] = pre_industrial_concentration['CH4']
cumulative_emissions['CH4'] = np.cumsum(emissions['CH4']) # cumulative emissions are not contributing to methane lifetime, but we'll track them in case future research makes them relevant

g0, g1 = calculate_g(lifetime['CH4'])

ch4_lifetime = np.ones(n_timesteps) * np.nan

for i in range(n_timesteps):
    alpha_lifetime = calculate_alpha(
        cumulative_emissions['CH4'][i],
        airborne_emissions,
        ssp585_temperature_rfmip[i],
        iirf_0["CH4"],
        iirf_cumulative["CH4"],
        iirf_temperature["CH4"],
        iirf_airborne["CH4"],
        g0,
        g1,
    )
    
    ch4_lifetime[i] = alpha_lifetime * lifetime['CH4']
    
    concentration['CH4'][i], gas_boxes, airborne_emissions = step_concentration(
        emissions['CH4'][i],
        gas_boxes,
        airborne_emissions,
        burden_per_emission['CH4'],
        lifetime['CH4'],
        alpha_lifetime=alpha_lifetime,
        pre_industrial_concentration=pre_industrial_concentration['CH4'],
        timestep=1,
        partition_fraction=partition_fraction['CH4']
    )

In [ ]:
for gas in gas_list:
    
    concentration[gas] = np.ones(n_timesteps) * np.nan
    gas_boxes = 0
    airborne_emissions = 0
    concentration[gas][0] = pre_industrial_concentration[gas]
    cumulative_emissions[gas] = np.cumsum(emissions[gas])

    if gas in ["CH4", "CO2"]:
        g0, g1 = calculate_g(lifetime[gas], partition_fraction=partition_fraction[gas])
        for i in range(n_timesteps):
            alpha_lifetime = calculate_alpha(
                cumulative_emissions[gas][i],
                airborne_emissions,
                ssp585_temperature_rfmip[i],
                iirf_0[gas],
                iirf_cumulative[gas],
                iirf_temperature[gas],
                iirf_airborne[gas],
                g0,
                g1,
            )
            concentration[gas][i], gas_boxes, airborne_emissions = step_concentration(
                emissions[gas][i], 
                gas_boxes,
                airborne_emissions, 
                burden_per_emission[gas],
                lifetime[gas],
                alpha_lifetime=alpha_lifetime,
                pre_industrial_concentration=pre_industrial_concentration[gas],
                timestep=1,
                partition_fraction=partition_fraction[gas],
                natural_emissions_adjustment=natural_emissions_adjustment[gas],
            )
    else:
        alpha_lifetime=1
        for i in range(n_timesteps):
            concentration[gas][i], gas_boxes, airborne_emissions = step_concentration(
                emissions[gas][i], 
                gas_boxes,
                airborne_emissions, 
                burden_per_emission[gas],
                lifetime[gas],
                alpha_lifetime=alpha_lifetime,
                pre_industrial_concentration=pre_industrial_concentration[gas],
                timestep=1,
                partition_fraction=partition_fraction[gas],
                natural_emissions_adjustment=natural_emissions_adjustment[gas],
            )

In [ ]:
effective_radiative_forcing = thornhill_skeie(
    emissions,
    concentration,
    temperature=ssp585_temperature_rfmip
)

In [ ]:
len(effective_radiative_forcing.keys())

In [ ]:
fig, ax = pl.subplots(3, 3, figsize=(16,16))
for icomp, component in enumerate(effective_radiative_forcing):
    iy = icomp % 3
    ix = icomp // 3
    ax[ix, iy].plot(np.arange(1750.5, 2501), effective_radiative_forcing[component], label='FaIR 2.1')
    ax[ix, iy].set_title(component)
fig.tight_layout()